In [3]:
!pip3 install albumentations

     |████████████████████████████████| 634kB 7.0MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654021 sha256=2e0df9702da165565526c7c7c000e694f4e0123bdd1a3e43c847e04049ff3e09
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


In [4]:
!pip install torchvision

In [5]:
from torch import nn
from loss import LossBinary
from torch.optim import Adam
from torch.utils.data import DataLoader
%matplotlib inline
from pylab import *
import cv2
import torch
from dataset_ft import load_image
from dataset_ft import load_mask
from utils import cuda
from generate_masks import get_model
from albumentations import Compose, Normalize
from albumentations.pytorch.functional import img_to_tensor
from validation import validation_binary
from dataset import RoboticsDataset
import glob
import matplotlib.pyplot as plt
from torch.utils.data.dataset import Dataset  # For custom data-sets
from torchvision import transforms
from torchsummary import summary
rcParams['figure.figsize'] = 10, 10

In [6]:
def img_transform(p=1):
    return Compose([
        Normalize(p=1)
    ], p=p)

In [7]:
def mask_overlay(image, mask, color=(0, 255, 0)):
    """
    Helper function to visualize mask on the top of the car
    """
    mask = np.dstack((mask, mask, mask)) * np.array(color)
    mask = mask.astype(np.uint8)
    weighted_sum = cv2.addWeighted(mask, 0.5, image, 0.5, 0.)
    img = image.copy()
    ind = mask[:, :, 1] > 0    
    img[ind] = weighted_sum[ind]    
    return img

In [8]:
model_path = 'data/models/linknet_binary_20/model_0.pt'
model = get_model(model_path, model_type='LinkNet34', problem_type='binary')

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


In [ ]:
from train_ft import train

jaccard_weight = 0.3

root = !pwd
root = root[0]
batch_size = 2
n_epochs = 20
lr1 = 0.0001
lr2 = 0.00001
num_workers = 12
train_size = 0.6
train_crop_height = 1056
train_crop_width = 1920
val_crop_height = 1056
val_crop_width = 1920
fold = "video9_raw"

train(model, "linknet", 0, "ent", jaccard_weight, fold, root, batch_size, n_epochs+10, lr1, num_workers, 1, train_size, train_crop_height, train_crop_width, val_crop_height, val_crop_width)
train(model, "linknet", 0, "ent", jaccard_weight, fold, root, batch_size, n_epochs+20, lr2, num_workers, 1, train_size, train_crop_height, train_crop_width, val_crop_height, val_crop_width)


num train = 1579, num_val = 1053


Epoch 21, lr 0.0001:   0%|          | 0/1580 [00:00<?, ?it/s]

Restored model, epoch 21, step 1,700


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
Epoch 22, lr 0.0001:   0%|          | 0/1580 [00:00<?, ?it/s]

Valid loss: 0.20432, jaccard: 0.66139


Epoch 22, lr 0.0001:  45%|████▍     | 706/1580 [11:49<14:28,  1.01it/s, loss=0.13956]